In [3]:
import anthropic
import os


client = anthropic.Anthropic(
    api_key = os.environ.get("ANTHROPIC_API_KEY")
)

[ContentBlock(text='Replit and Streamlit are both platforms for building and hosting web apps in Python, but have some key differences:\n\n- Hosting: Replit hosts apps on its cloud platform and provides a shareable URL. Streamlit apps have to be self-hosted somewhere like Heroku. Streamlit does offer a paid Streamlit Cloud hosting service though.\n\n- Interface: Streamlit uses Python for both writing logic and UI layout. Replit uses Python + traditional HTML/CSS/JS.\n\n- Usage: Streamlit is focused specifically on creating Python data apps and dashboards. Replit supports full stack web apps with databases etc.\n\n- Models/Computation: Streamlit runs Python code locally when developing. For hosting, it', type='text')]


In [49]:
system_prompt = """you are a very competent teacher

you are teaching year 13 A-level students

read the syllabus carefully
"""

# user_prompt_pre = """
# <syllabus> </syllabus>
# create questions based on these key concepts that will test student’s understanding

# put each question in question tags

# give the answers also in answer tags

# each answer after each question
# """

user_prompt_pre = """
<syllabus> </syllabus>
create clear questions from the syllabus that will test student’s understanding

the answers will be clear and elaborate

put each question in question tags

give the answers in answer tags

Skip the preamble in the response.
"""

In [8]:
content = """
Aristotle (384 B.C.E.—322 B.C.E.)
Aristotle is a towering figure in ancient Greek philosophy, who made important contributions to logic, criticism, rhetoric, physics, biology, psychology, mathematics, metaphysics, ethics, and politics. He was a student of Plato for twenty years but is famous for rejecting Plato’s theory of forms. He was more empirically minded than both Plato and Plato’s teacher, Socrates.

A prolific writer, lecturer, and polymath, Aristotle radically transformed most of the topics he investigated. In his lifetime, he wrote dialogues and as many as 200 treatises, of which only 31 survive. These works are in the form of lecture notes and draft manuscripts never intended for general readership. Nevertheless, they are the earliest complete philosophical treatises we still possess.

As the father of western logic, Aristotle was the first to develop a formal system for reasoning. He observed that the deductive validity of any argument can be determined by its structure rather than its content, for example, in the syllogism: All men are mortal; Socrates is a man; therefore, Socrates is mortal. Even if the content of the argument were changed from being about Socrates to being about someone else, because of its structure, as long as the premises are true, then the conclusion must also be true. Aristotelian logic dominated until the rise of modern propositional logic and predicate logic 2000 years later.

The emphasis on good reasoning serves as the backdrop for Aristotle’s other investigations. In his natural philosophy, Aristotle combines logic with observation to make general, causal claims. For example, in his biology, Aristotle uses the concept of species to make empirical claims about the functions and behavior of individual animals. However, as revealed in his psychological works, Aristotle is no reductive materialist. Instead, he thinks of the body as the matter, and the psyche as the form of each living animal.

Though his natural scientific work is firmly based on observation, Aristotle also recognizes the possibility of knowledge that is not empirical. In his metaphysics, he claims that there must be a separate and unchanging being that is the source of all other beings. In his ethics, he holds that it is only by becoming excellent that one could achieve eudaimonia, a sort of happiness or blessedness that constitutes the best kind of human life.

Aristotle was the founder of the Lyceum, a school based in Athens, Greece; and he was the first of the Peripatetics, his followers from the Lyceum. Aristotle’s works, exerted tremendous influence on ancient and medieval thought and continue to inspire philosophers to this day.
"""
user_prompt = user_prompt_pre + content

In [18]:
message = client.messages.create(
    model="claude-2.1",
    max_tokens=1500,
    temperature=1,
    messages=[{"role": "system", 
               "content": system_prompt,
               "role": "user",
               "content": user_prompt
              }]    
)
print(message.content[0].text)

[ContentBlock(text="Here are some potential exam questions on the key concepts in Aristotle's philosophy:\n\n<question>What is Aristotle's view on Plato's theory of forms?</question>\n<answer>Aristotle rejected Plato's theory of forms, instead being more empirically minded.</answer>\n\n<question>What formal system for reasoning did Aristotle develop?</question>  \n<answer>Aristotle developed formal logic and was the first to develop a system for assessing deductive arguments based on their structure rather than content.</answer>\n\n<question>What is an example of a deductive syllogism Aristotle discusses?</question>\n<answer>An example is: All men are mortal; Socrates is a man; therefore, Socrates is mortal.</answer>\n\n<question>How did Aristotle combine logic and observation in his natural philosophy?</question>\n<answer>Aristotle used logic to make general, causal claims based on empirical observations in domains like biology.</answer>  \n\n<question>What are the key concepts in Ari

In [20]:
from dataclasses import dataclass
from typing import List
import xml.etree.ElementTree as ET

@dataclass
class Question:
    question: str
    answer: str

def parse_questions(xml_string: str) -> List[Question]:
    questions = []
    
    # Parse the XML string
    root = ET.fromstring(f"<root>{xml_string}</root>")  # Wrapping with a root tag since XML needs a single root element
    
    # Iterate over question elements
    for question_elem in root.findall('question'):
        # Find the corresponding answer element
        answer_elem = question_elem.find('answer')
        question_text = question_elem.text.strip() if question_elem.text is not None else ""
        answer_text = answer_elem.text.strip() if answer_elem.text is not None else ""
        
        # Create a Question instance and append to the list
        questions.append(Question(question=question_text, answer=answer_text))
    
    return questions

In [43]:
from dataclasses import dataclass
import re

# Define the Question dataclass
@dataclass
class Question:
    question: str
    answer: str

def parse_questions(input_string: str) -> list[Question]:
    """
    Parses a string containing question and answer pairs in a custom XML format 
    and returns a list of Question objects.
    
    Parameters:
    - input_string: str. The input string in a custom XML format.
    
    Returns:
    - A list of Question objects extracted from the input string.
    """
    # Regular expression to find all question and answer pairs
    pattern = re.compile(r'<question>(.*?)<\/question>\s*<answer>(.*?)<\/answer>')
    
    # Find all matches of the pattern in the input string
    matches = pattern.findall(input_string)
    
    # Instantiate Question objects for each found pair and collect them in a list
    questions = [Question(question=match[0], answer=match[1]) for match in matches]
    
    return questions

# Example usage
if __name__ == "__main__":
    input_string = """<question>Some text for one question</question>.
<answer>Some text for an answer</answer>
<question>Some text for one question</question>.
<answer>Some text for an answer</answer>
"""
    questions = parse_questions(input_string)
    for q in questions:
        print(f"Question: {q.question}, Answer: {q.answer}")

In [47]:
# Prompt
qlist = parse_questions(message.content[0].text)
for q in qlist:
        print(f"Question: {q.question}\nAnswer: {q.answer}\n\n")

Question: What is Aristotle's view on Plato's theory of forms?
Answer: Aristotle rejected Plato's theory of forms, instead being more empirically minded.


Question: What formal system for reasoning did Aristotle develop?
Answer: Aristotle developed formal logic and was the first to develop a system for assessing deductive arguments based on their structure rather than content.


Question: What is an example of a deductive syllogism Aristotle discusses?
Answer: An example is: All men are mortal; Socrates is a man; therefore, Socrates is mortal.


Question: How did Aristotle combine logic and observation in his natural philosophy?
Answer: Aristotle used logic to make general, causal claims based on empirical observations in domains like biology.


Question: What are the key concepts in Aristotle's metaphysics?
Answer: Aristotle argued that there must be a separate, unchanging being that is the source of all other beings.


Question: According to Aristotle, how does one achieve eudaimoni